<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Extract-all-routes" data-toc-modified-id="Extract-all-routes-1">Extract all routes</a></span></li></ul></div>

In [2]:
import os
import h5py
from bfseg.data.meshdist.dataLoader import DataLoader
import pandas as pd
import re
import os
from bfseg.utils.utils import dump_meshdist_ds_to_h5

In [83]:
train_path = "/home/rene/cla_dataset/fused/"
validation_path = '/home/rene/hiveLabels/'

In [84]:
dl_cla = DataLoader(train_path,
                     [480, 640],
                     validationDir=validation_path,
                     validationMode='CLA',
                     batchSize=4, loadDepth=False)

In [85]:
ds_cla, _ = dl_cla.getExportableDataset()

## Extract all routes

In [92]:
regex_pattern = "(.*)_img_(\d+)_semseg\.png"
all_labels = dl_cla.labels

In [93]:
all_routes = set([re.search(regex_pattern, os.path.basename(label)).group(1) for label in all_labels])

In [94]:
dataset_information = {}

In [95]:
for route in all_routes:
    print(route)
    info_file = os.path.join(train_path, f"{route}_info.txt")
    dataset_information[route] = pd.read_csv(info_file, header=None, sep=',|;', engine="python")

cam0
cam1
cam2


In [96]:
img_timestamps = []
for label in all_labels:
    match = re.search(regex_pattern, os.path.basename(label))
    route, number = match.group(1), int(match.group(2))
    timestamp = dataset_information[route].iloc[number,1]
    img_timestamps.append(route +"_"+ str(timestamp))

In [97]:
datasets = [("CLA_FUSED", ds_cla, img_timestamps)]

In [98]:
dump_meshdist_ds_to_h5(datasets, dump_depth = False)